# Normalizing the json file

In [57]:
import pandas as pd
import json
from pandas import json_normalize

# Load JSON data line by line and convert it into a list of dictionaries
data = []
with open('C:/Users/lelee/Desktop/Project3/output.json', 'r') as file:
    for line in file:
        try:
            json_object = json.loads(line)
            # Convert 'saleDate' to datetime object
            json_object['saleDate'] = pd.to_datetime(json_object['saleDate']['$date'])
            data.append(json_object)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Normalize the 'items' column into a new DataFrame
items_df = json_normalize(data, 'items', ['_id', 'saleDate', 'storeLocation', 'customer', 'couponUsed', 'purchaseMethod'])

# Perform your analysis on the normalized items DataFrame
top_products = items_df.groupby('name')['quantity'].sum().sort_values(ascending=False).head(10)
print(top_products)

name
binder           25493
envelopes        25078
notepad          20727
pens             13766
printer paper    12092
backpack          6918
laptop            6793
Name: quantity, dtype: int64


# Printing column name

In [58]:
print(items_df.columns)



Index(['name', 'tags', 'quantity', 'price.$numberDecimal', '_id', 'saleDate',
       'storeLocation', 'customer', 'couponUsed', 'purchaseMethod'],
      dtype='object')


# Show top 10 products (name) sales (quantity x price). 

In [59]:
# Convert 'price.$numberDecimal' column to numeric
items_df['price.$numberDecimal'] = pd.to_numeric(items_df['price.$numberDecimal'], errors='coerce')

# Calculate total sales for each product (quantity x price)
items_df['total_sales'] = items_df['quantity'] * items_df['price.$numberDecimal']

# Group by product name and calculate total sales, then sort and get the top 10
top_products_sales = items_df.groupby('name')['total_sales'].sum().sort_values(ascending=False).head(10)
print(top_products_sales)


name
laptop           6775977.07
backpack          817374.10
pens              581843.27
binder            511644.57
notepad           463615.48
envelopes         376658.49
printer paper     367459.29
Name: total_sales, dtype: float64


# Show top 3 products (name) sales by store (location). 

In [60]:
# Group by store location, product name, and calculate total sales, then sort and get the top 3 for each location
top_products_by_location = items_df.groupby(['storeLocation', 'name'])['total_sales'].sum().reset_index()
top_products_by_location = top_products_by_location.groupby('storeLocation').apply(lambda x: x.nlargest(3, 'total_sales')).reset_index(drop=True)

print(top_products_by_location)


   storeLocation      name  total_sales
0         Austin    laptop   1018494.05
1         Austin  backpack    115217.59
2         Austin      pens     74622.52
3         Denver    laptop   1961659.25
4         Denver  backpack    245679.34
5         Denver      pens    185008.39
6         London    laptop   1073098.72
7         London  backpack    144529.14
8         London      pens     92602.30
9       New York    laptop    694613.32
10      New York  backpack     82563.14
11      New York      pens     64029.90
12     San Diego    laptop    462973.58
13     San Diego  backpack     50997.42
14     San Diego      pens     42134.55
15       Seattle    laptop   1565138.15
16       Seattle  backpack    178387.47
17       Seattle      pens    123445.61


# Show rankings of each store (location). 

In [61]:
# Group by store location and calculate total sales, then rank the stores based on total sales
store_rankings = items_df.groupby('storeLocation')['total_sales'].sum().sort_values(ascending=False).reset_index()
store_rankings['rank'] = store_rankings['total_sales'].rank(ascending=False).astype(int)

print(store_rankings)


  storeLocation  total_sales  rank
0        Denver   2921009.92     1
1       Seattle   2255947.69     2
2        London   1583066.79     3
3        Austin   1445603.11     4
4      New York   1016059.59     5
5     San Diego    672885.17     6


# Show purchased method by gender table 

In [62]:
# Extract information from the 'customer' column
items_df['gender'] = items_df['customer'].apply(lambda x: x.get('gender'))
items_df['age'] = items_df['customer'].apply(lambda x: x.get('age'))
items_df['email'] = items_df['customer'].apply(lambda x: x.get('email'))
items_df['satisfaction'] = items_df['customer'].apply(lambda x: x.get('satisfaction'))

# Drop the original 'customer' column
items_df.drop(columns=['customer'], inplace=True)

In [63]:
# Clean up column names by stripping extra spaces
items_df.columns = items_df.columns.str.strip()

In [64]:
# Replace 'M' with 'Male' and 'F' with 'Female' in the 'gender' column
items_df['gender'] = items_df['gender'].replace({'M': 'Male', 'F': 'Female'})

# Show purchase method by gender table
purchase_method_by_gender = items_df.groupby(['purchaseMethod', 'gender']).size().reset_index(name='count')
purchase_method_by_gender_pivot = purchase_method_by_gender.pivot(index='gender', columns='purchaseMethod', values='count')

# Reorder the rows to have 'Male' first
purchase_method_by_gender_pivot = purchase_method_by_gender_pivot.reindex(index=['Male', 'Female'])

# Display the updated purchase method by gender table
print(purchase_method_by_gender_pivot)


purchaseMethod  In store  Online  Phone
gender                                 
Male                7571    4314   1722
Female              8062    4270   1499


# Show monthly total sales 

In [65]:
# Calculate monthly total sales
items_df['YearMonth'] = items_df['saleDate'].dt.to_period('M')
monthly_sales = items_df.groupby('YearMonth')['total_sales'].sum()
print(monthly_sales)

YearMonth
2013-01    196043.76
2013-02    147622.59
2013-03    140700.06
2013-04    162336.67
2013-05    179143.37
2013-06    142132.02
2013-07    176298.23
2013-08    130977.22
2013-09    137871.99
2013-10    173126.73
2013-11    148358.38
2013-12    174306.96
2014-01    125824.02
2014-02    141174.41
2014-03    193413.50
2014-04    132593.08
2014-05    166896.80
2014-06    143304.56
2014-07    184804.60
2014-08    167979.81
2014-09    159802.92
2014-10    149284.84
2014-11    180564.85
2014-12    139466.83
2015-01    222577.01
2015-02    189776.15
2015-03    178086.94
2015-04    132298.50
2015-05    167482.69
2015-06    149484.34
2015-07    149510.47
2015-08    154868.63
2015-09    135615.80
2015-10    175215.34
2015-11    158390.42
2015-12    166565.02
2016-01    166558.93
2016-02    120783.52
2016-03    193185.17
2016-04    137340.27
2016-05    162966.06
2016-06    132942.54
2016-07    180331.10
2016-08    164555.33
2016-09    138299.53
2016-10    151698.93
2016-11    170758.41
201

C:\Users\lelee\AppData\Local\Temp\ipykernel_2232\1649362248.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  items_df['YearMonth'] = items_df['saleDate'].dt.to_period('M')
